In [5]:

from __future__ import absolute_import
from __future__ import print_function

import os
import csv
import six

import numpy as np
import time
import json
import warnings

from collections import deque
from collections import OrderedDict
from collections import Iterable

# os.environ["THEANO_FLAGS"] = "device=gpu,floatX=float32" 
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=gpu0,floatX=float32,force_device=true,exception_verbosity=high,lib.cnmem=0.9 ,nvcc.flags=-D_FORCE_INLINES"
# os.environ[
#     "THEANO_FLAGS"] = "mode=FAST_RUN,device=cuda,floatX=float32,force_device=true,nvcc.flags=-D_FORCE_INLINES"

os.environ['KERAS_BACKEND'] = 'theano'

import sys
import numpy as np
from keras import backend as K
K.set_image_dim_ordering('th')
# K.set_image_data_format('channels_first')
#from keras.datasets import cifar10
#keras.callbacks.Callback()
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Convolution2D, MaxPooling2D 
from keras.layers import GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization as BN
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import model_from_json
from keras.callbacks import ModelCheckpoint, Callback
from keras.datasets import mnist
from scipy import io
from IPython.core.debugger import Tracer
import matplotlib.pyplot as plt
from keras import metrics
import numpy as np

import keras.backend as K

Using Theano backend.


AttributeError: 'module' object has no attribute 'tests'

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.debugger import Tracer



In [ ]:
def model_architecture(img_rows,img_cols,img_channels,nb_classes):
    #function defining the architecture of defined CNN
    
    model = Sequential()

    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True, input_shape=(img_channels,img_rows, img_cols)))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(32, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))
    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))
    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.25))

    model.add(Convolution2D(96, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(MaxPooling2D(pool_size=(2, 2), strides = (2,2)))
    model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.5))

    model.add(Convolution2D(512, 1, 1, activation='relu', border_mode='same',init='orthogonal', bias = True))

    model.add(BN(axis=1, momentum=0.99, epsilon=0.001))
    Dropout((0.5))

    model.add(Convolution2D(2, 1, 1, activation='relu', border_mode='same',init='orthogonal', bias = True))
    model.add(GlobalAveragePooling2D(dim_ordering='default'))

    model.add(Activation('softmax'))
#     model.summary()
    return model


def normalize_date(X,Y,nb_classes):
    X = X.astype('float32')
    Y = Y.astype('float32')

#     normalizing the data
    X /= 4095.0   
    Y = np_utils.to_categorical(Y, nb_classes)
    return X,Y


In [ ]:
def get_categorical_accuracy_keras(y_true, y_pred):
    return K.mean(K.equal(K.argmax(y_true, axis=1), K.argmax(y_pred, axis=1)))

In [ ]:
nb_classes = 2
img_rows, img_cols = 96,96
img_channels = 1

decon = np.load('/work/vsankar/Project-Luna/data/lung_decon/binary_heatmap_output_val_data_net_val_acc_92.5.npy')
X = np.load('/work/vsankar/Project-Luna/data/lung_decon/Val_nf_n_0_rotated_data.npz_FILES/X_val.npy')
Y = np.load('/work/vsankar/Project-Luna/data/lung_decon/Val_nf_n_0_rotated_data.npz_FILES/Y_val.npy')

X,Y = normalize_date(X,Y,nb_classes)

model_name = 'luna_weights_t_ae5_nt_b128_d25_BN1_d3_3_rotated_best_dev_new'
modelPath = '/work/vsankar/Project-Luna/data/lung_decon/best_weights_test_acc_92.5/'+model_name+'.hdf5'

model = model_architecture(img_rows,img_cols,img_channels,nb_classes)
model.load_weights(modelPath)

# training the model using SGD + momentum
adm = Adam(lr=0.000001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(loss='categorical_crossentropy',
              optimizer=adm,
              metrics=[get_categorical_accuracy_keras])

In [ ]:
decon.shape

In [ ]:
len(Y[np.where(Y[:,0]==0)])

In [ ]:
len(Y[np.where(Y[:,0]==1)])

In [ ]:
Y[1]

In [ ]:
X.shape

In [ ]:
# score = model.evaluate(X,Y,verbose=0)
# print (score)
# print ('test_accuracy is : ', score[1])
pred = model.predict(X)
y_pred = []
y_true_label =[]
for p in pred:
    y_pred.append(np.argmax(p))
for l in Y:
    y_true_label.append(np.argmax(l))

# print (classification_report(y_true_label,y_pred))
# print (roc_auc_score(y_true_label,y_pred))

In [ ]:
deconX = np.zeros_like(decon)
for idx in range(len(Y)):
    # multiply
    deconX[idx,0,0,:,:] = np.multiply(X[idx,0,:,:],decon[idx,0,0,:,:])
    deconX[idx,1,0,:,:] = np.multiply(X[idx, 0, :, :], decon[idx, 1, 0, :, :])
  
    y_predict = model.predict(X[idx:idx+1,:,:,:])
#     Tracer()()
    plt.subplot(1, 3, 1)
    plt.imshow(X[idx,0,:,:])
    plt.title('X'+' Y = '+ str(int(Y[idx])))

    plt.subplot(1,3,2)
    plt.imshow(decon[idx,0,0,:,:])
    plt.title('d0 Y_p=' + str(int(y_predict[0][0])))

    plt.subplot(1,3,3)
    plt.imshow(decon[idx,1,0,:,:])
    plt.title('d1 Y_p=' + str(int(y_predict[0][0])))
    plt.show()
    
#     Tracer()()
    savefigure = '/work/vsankar/Project-Luna/data/lung_decon/images/decon_comparison_' + str(idx) + '.png'
    plt.savefig(savefigure)
#     Tracer()()
    a=1
np.savez('/work/vsankar/Project-Luna/data/lung_decon/deconX',deconX =  deconX)

In [ ]:
plt.imshow(deconX[3000,1,0,:,:])


In [ ]:
np.savez('/work/vsankar/Project-Luna/Train_all_dA_0for9k_1for0_rotated_data',X_train =  X_train,Y_train = Y_train)

In [ ]:
plt.subplot(1,3,1)
plt.imshow(X[idx,0,:,:])

plt.subplot(1,3,2)
plt.imshow(decon[idx,0,0,:,:])

plt.subplot(1,3,3)
plt.imshow(decon[idx,1,0,:,:])
plt.show()